In [4]:
# Bibliotecas
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity
from tqdm.auto import tqdm
import re
import nltk

# Configuración de NLTK
for recurso in ['punkt', 'stopwords']:
    nltk.download(recurso)
palabras_vacias = set(stopwords.words('english'))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\galot\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\galot\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
# Configuración para web scraping
URL_BASE = "https://www.allrecipes.com/recipes-a-z-6735880"
CABECERAS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/131.0.0.0",
    "Accept": "text/html,application/xml;q=0.9,image/webp,*/*;q=0.8",
    "Accept-Language": "es-ES,es;q=0.8,en-US;q=0.5,en;q=0.3",
    "Connection": "keep-alive",
    "Upgrade-Insecure-Requests": "1"
}

In [6]:
def extraer_recetas():
    """Extrae las recetas del sitio web y retorna un DataFrame"""
    recetas_datos = []
    
    # Obtener página principal
    respuesta = requests.get(URL_BASE, headers=CABECERAS)
    
    if respuesta.status_code != 200:
        raise Exception(f"Error de acceso: {respuesta.status_code}")
        
    sopa = BeautifulSoup(respuesta.text, 'html.parser')
    elementos_principales = sopa.find_all('li', class_='mntl-link-list__item')
    
    for elemento in tqdm(elementos_principales, desc="Extrayendo recetas"):
        enlace = elemento.find('a')
        if not enlace:
            continue
            
        categoria = enlace.get_text(strip=True)
        url_categoria = enlace['href']
        
        # Procesar página de categoría
        resp_categoria = requests.get(url_categoria, headers=CABECERAS)
        if resp_categoria.status_code == 200:
            sopa_categoria = BeautifulSoup(resp_categoria.text, 'html.parser')
            recetas = sopa_categoria.find_all('a', class_='comp mntl-card-list-items mntl-universal-card mntl-document-card mntl-card card card--no-image')
            
            for receta in recetas:
                recetas_datos.append({
                    'Categoria': categoria,
                    'URL_Categoria': url_categoria,
                    'Nombre_Receta': receta.get_text(strip=True),
                    'URL_Receta': receta.get('href', '')
                })
    
    return pd.DataFrame(recetas_datos)

# Extracción
df_recetas = extraer_recetas()
df_recetas = df_recetas[:600]  

Extrayendo recetas:   0%|          | 0/378 [00:00<?, ?it/s]

In [7]:
def extraer_detalles_receta(url):
    """
    Extrae los detalles específicos de una receta
    """
    try:
        respuesta = requests.get(url, headers=CABECERAS)
        respuesta.raise_for_status()
        sopa = BeautifulSoup(respuesta.content, 'html.parser')
        
        # Extraer nombre
        titulo = sopa.find('h1', class_='article-heading text-headline-400')
        nombre = titulo.get_text(strip=True) if titulo else 'Sin título'
        
        # Extraer ingredientes
        lista_ingredientes = []
        elementos_ingredientes = sopa.find_all('li', class_='mm-recipes-structured-ingredients__list-item')
        
        for elemento in elementos_ingredientes:
            partes = {
                'cantidad': elemento.find('span', {'data-ingredient-quantity': 'true'}),
                'unidad': elemento.find('span', {'data-ingredient-unit': 'true'}),
                'ingrediente': elemento.find('span', {'data-ingredient-name': 'true'})
            }
            
            ingrediente_completo = ' '.join(
                parte.get_text(strip=True) if parte else ''
                for parte in partes.values()
            ).strip()
            
            if ingrediente_completo:
                lista_ingredientes.append(ingrediente_completo)
                
        # Extraer preparación
        pasos_preparacion = []
        seccion_preparacion = sopa.find('ol', class_='comp mntl-sc-block mntl-sc-block-startgroup mntl-sc-block-group--OL')
        
        if seccion_preparacion:
            pasos_preparacion = [paso.get_text(strip=True) 
                               for paso in seccion_preparacion.find_all('li')]
        
        return {
            'nombre': nombre,
            'ingredientes': ', '.join(lista_ingredientes),
            'preparacion': ' '.join(pasos_preparacion)
        }
        
    except Exception as error:
        print(f"Error en URL {url}: {str(error)}")
        return {
            'nombre': 'Error',
            'ingredientes': 'Error',
            'preparacion': 'Error'
        }


def actualizar_recetas(df):
    """
    Actualiza el DataFrame con los detalles de cada receta
    """
    for indice in tqdm(df.index, desc="Procesando recetas"):
        detalles = extraer_detalles_receta(df.loc[indice, 'URL_Receta'])
        for campo, valor in detalles.items():
            df.loc[indice, campo.capitalize()] = valor
    
    return df

# Ejecutar 
df_recetas = actualizar_recetas(df_recetas)
df_recetas.to_excel('recetas_completasx.xlsx', index=False)

Procesando recetas:   0%|          | 0/600 [00:00<?, ?it/s]

In [8]:
class ProcesadorRecetas:
    def __init__(self):
        self.vectorizador = TfidfVectorizer(max_features=1000)
        self.modelo_clustering = KMeans(n_clusters=10, random_state=42)
        
    def limpiar_texto(self, texto):
        """Preprocesa y limpia el texto"""
        # Convertir a minúsculas y eliminar caracteres especiales
        texto = texto.lower()
        texto = re.sub(r'[^a-zA-Z]', ' ', texto)
        
        # Tokenización y eliminación de palabras vacías
        tokens = word_tokenize(texto)
        tokens_filtrados = [palabra for palabra in tokens 
                          if palabra not in palabras_vacias and len(palabra) > 2]
        
        return ' '.join(tokens_filtrados)
    
    def preparar_datos(self, df):
        """Prepara los datos para el análisis"""
        # Combinar textos y limpiar
        df['texto_completo'] = df['Ingredientes'].fillna('') + ' ' + df['Preparacion'].fillna('')
        df['texto_procesado'] = df['texto_completo'].apply(self.limpiar_texto)
        
        # Generar embeddings
        self.embeddings = self.vectorizador.fit_transform(df['texto_procesado'])
        
        # Realizar clustering
        df['cluster'] = self.modelo_clustering.fit_predict(self.embeddings)
        
        return df

# Inicializar procesador y ejecutar
procesador = ProcesadorRecetas()
df_recetas = procesador.preparar_datos(df_recetas)

In [10]:
class BuscadorRecetas:
    def __init__(self, procesador, df):
        self.procesador = procesador
        self.df = df
    
    def buscar_similares(self, consulta, n_resultados=10):
        """Busca recetas similares a la consulta"""
        consulta_limpia = self.procesador.limpiar_texto(consulta)
        consulta_vector = self.procesador.vectorizador.transform([consulta_limpia])
        similitudes = cosine_similarity(consulta_vector, self.procesador.embeddings).flatten()
        indices_mejores = similitudes.argsort()[-n_resultados:][::-1]
        
        resultados = []
        for idx in indices_mejores:
            receta = self.df.iloc[idx]
            resultados.append({
                'nombre': receta['Nombre'],
                'similitud': similitudes[idx],
                'cluster': receta['cluster'],
                'ingredientes': receta['Ingredientes'],
                'preparacion': receta['Preparacion'],
                'url': receta['URL_Receta']
            })
        return resultados

    def formatear_receta(self, receta, indice):
        """Formatea una receta individual con estilo ASCII art"""
        similitud_porcentaje = receta['similitud'] * 100
        barra_similitud = '█' * int(similitud_porcentaje/10) + '░' * (10 - int(similitud_porcentaje/10))
        
        formato = f"""
╔══════════════════════ RECETA {indice} ══════════════════════╗
║ {receta['nombre']:<48} ║
╠══════════════════════════════════════════════════════════════╣
║ Similitud: [{barra_similitud}] {similitud_porcentaje:.1f}%
║ Grupo de recetas: {receta['cluster']}
║ 
║ 🥘 INGREDIENTES:
"""
        # Formatear ingredientes
        ingredientes = receta['ingredientes'].split(',')
        for ing in ingredientes:
            if ing.strip():
                formato += f"║   • {ing.strip()[:45]:<45} ║\n"
        
        formato += "║\n║ 👩‍🍳 PREPARACIÓN:\n"
        
        # Formatear pasos de preparación
        pasos = receta['preparacion'].split('.')
        for i, paso in enumerate(pasos, 1):
            if paso.strip():
                # Dividir pasos largos en múltiples líneas
                palabras = paso.strip().split()
                linea_actual = f"║   {i}. "
                for palabra in palabras:
                    if len(linea_actual + palabra) > 45:
                        formato += f"{linea_actual:<50} ║\n"
                        linea_actual = f"║      {palabra}"
                    else:
                        linea_actual += f"{palabra} "
                if linea_actual.strip():
                    formato += f"{linea_actual:<50} ║\n"
        
        formato += "║\n║ 🔗 URL: " + receta['url'][:45] + "\n"
        formato += "╚══════════════════════════════════════════════════════════════╝"
        return formato

    def mostrar_resultados(self, consulta):
        """Muestra los resultados con formato personalizado"""
        resultados = self.buscar_similares(consulta)
        print(f"\n🔍 Búsqueda: '{consulta}'\n")
        print("📊 Encontradas las siguientes recetas similares:\n")
        
        for i, resultado in enumerate(resultados, 1):
            print(self.formatear_receta(resultado, i))
            if i < len(resultados):
                print("\n" + "▫" * 60 + "\n")

# Crear buscador y ejecutar ejemplo
buscador = BuscadorRecetas(procesador, df_recetas)
buscador.mostrar_resultados("pan de chocolate")


🔍 Búsqueda: 'pan de chocolate'

📊 Encontradas las siguientes recetas similares:


╔══════════════════════ RECETA 1 ══════════════════════╗
║ White Chocolate Cinnamon Toast Crunch Bars       ║
╠══════════════════════════════════════════════════════════════╣
║ Similitud: [█████░░░░░] 55.8%
║ Grupo de recetas: 9
║ 
║ 🥘 INGREDIENTES:
║   • 3 1/4 cups white chocolate chips              ║
║   • 2 teaspoons ground cinnamon                   ║
║   • 1 cup innamon Toast Crunch                    ║
║
║ 👩‍🍳 PREPARACIÓN:
║   1. Melt 3 cups white chocolate in a            ║
║      doubleboiler or place in a                  ║
║      microwave-safebowl and microwave in         ║
║      30-secondintervals, stirring between        ║
║      intervalsuntil melted                       ║
║   2. Add cinnamon; stir to combine               ║
║   3. In a 12-well silicone snack bar pan:        ║
║      Place3-6 cereal pieces in each well         ║
║   4. Spoon chocolate over cereal                 ║
║   5. 